In [7]:
import psycopg2
import json

# Database connection parameters
db_params = {
    "dbname": "mydatabase",
    "user": "myuser",
    "password": "mypassword",
    "host": "localhost",  # or the appropriate hostname of your db server
    "port": "5432"
}

# READ JSON file
json_file_path = 'formatted_data.json'
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Connect to the database
conn = psycopg2.connect(**db_params)
cur = conn.cursor()

# create table
create_table_query = """
    CREATE TABLE IF NOT EXISTS cryptocurrency_data (
        timestamp TIMESTAMP,
        place INT,
        name VARCHAR(255),
        price DECIMAL,
        volume DECIMAL,
        top_tier_volume DECIMAL,
        market_cap DECIMAL,
        percentage_change DECIMAL
    );
"""
cur.execute(create_table_query)

# Insert data
insert_query = """
    INSERT INTO cryptocurrency_data 
    (timestamp, place, name, price, volume, top_tier_volume, market_cap, percentage_change) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

for entry in data["data"]:
    cur.execute(insert_query, (
        data["timestamp"],
        entry["place"],
        entry["name"],
        entry["price"],
        entry["volume"],
        entry["top_tier_volume"],
        entry["market_cap"],
        entry["percentage_change"]
    ))

# Commit changes and close the connection
conn.commit()
cur.close()
conn.close()

In [ ]:
def delete_table(table_name):
    # Connect to the database
    conn = psycopg2.connect(**db_params)
    cur = conn.cursor()

    # Delete table
    try:
        cur.execute(f"DROP TABLE IF EXISTS {table_name};")
        print(f"Table {table_name} deleted successfully.")
        conn.commit()
    except Exception as e:
        print(f"An error occurred: {e}")
        conn.rollback()
    finally:
        # Close the connection
        cur.close()
        conn.close()

# Replace 'cryptocurrency_data' with your table name
delete_table("cryptocurrency_data")